<p style="background-color:darkblue;font-family:arial;font-size:250%;color:white;text-align:center;border-radius:16px 16px;"><b>Detection of product defects using Yolov7</b></p>

### Ruthger Righart
#### rrighart@googlemail.com

Product packaging defects can disturb the supply chain and lead to customer insatisfaction. A wide variety of packaging problems can occur in the production line such as label issues, bad seals, damage to jar lids, such as scratches, deformations and holes. Product defects could cause contamination of food or other substances. It is therefore crucial to detect them without delay.

In [ ]:
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
img = Image.open('/kaggle/input/jarlids/p20.JPG')
i1 = ImageDraw.Draw(img)
font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 16, encoding="unic")

i1.text((8, 8), "Examples of jar lid damage", font=font, fill=(255, 255, 255))
img

Computer vision can be used to detect such defects. It is particularly suited for this purpose since it is fast and automated (little human assistance is needed) <span style="color:blue">[1]</span>.

The current notebook is experimental and can be regarded as proof of concept. It examines if jar lid defects can be detected using Yolov7. Yolo (You Only Look Once) is an algorithm that has been used successfully for object detection in various interesting domains, such as the inspection of metal surface defects <span style="color:blue">[2]</span>.

<a id="Table"><b>Table of contents:</b>
<ul>
<li><a href="#Installation-packages">Installation packages</a></li>  
<li><a href="#Image-annotations">Image data and annotations</a></li>         
<li><a href="#Data-preprocessing">Data preprocessing</a></li>
<li><a href="#Visualization">Visualization and annotation boxes</a></li>
<li><a href="#Bounding-box">Bounding box size</a></li>
<li><a href="#Class-distribution">Class distribution</a></li>
<li><a href="#Preparation-annotations">Preparation of annotations</a></li>
<li><a href="#Training">Training</a></li>
<li><a href="#Visualization-learning">Visualization of learning curves</a></li>
<li><a href="#Unseen-testdata">Unseen testdata</a></li>
<li><a href="#Closing-words">Closing words</a></li>
    
</ul>


There is an App where you can interact with the final model. You can upload your own images and see the predicted classes.

👉 https://huggingface.co/spaces/rrighart/product-defects

**Yolo**

In Yolo, object detection is framed as a regression problem where a single neural network predicts bounding boxes and class probabilities directly from the presented image files. Yolo sees the entire image during training so it implicitly encodes contextual information (unlike sliding window and region proposal-based techniques) <span style="color:blue">[3]</span>.

Yolov7 outperforms other object detectors in terms of accuracy and speed. It uses model reparameterization and model scaling <span style="color:blue">[4]</span>.

![](https://github.com/WongKinYiu/yolov7/raw/main/figure/performance.png)

<a id="Installation-packages"></a>
<p style="background-color:darkblue;font-family:arial;font-size:120%;color:white;text-align:center;border-radius:2px 2px;"><b>Installation packages</b></p>

We start with installing the needed packages. Do not forget to set the accelerator in the right panel at GPU.

In [ ]:
!git clone https://github.com/WongKinYiu/yolov7 
!cd yolov7 && wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

In [ ]:
!pip install imagesize

In [ ]:
!pip install ptitprince

The following modules will be used:

In [ ]:
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import json
import sys
import os
import shutil
from distutils.dir_util import copy_tree
import imagesize
import ptitprince as pt
from shutil import copyfile
import time
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.patches as mpatches
import seaborn as sns
import re

Yolov7 is setting the seed automatically at 0 for `random.seed`, `np.random.seed` and `init_torch_seeds` in the function init_seeds that can be found in `yolov7/utils/general.py`. Note however that complete reproducibility across algorithm releases and CPU/GPU is not garanteed.

In [ ]:
!grep -w init_seeds 'yolov7/utils/general.py' -A 4 

<a id="Image-data"></a>
<p style="background-color:darkblue;font-family:arial;font-size:120%;color:white;text-align:center;border-radius:2px 2px;"><b>Image data and annotations</b></p>

Photos were taken using a Panasonic camera (DMC-TZ7). All images were width 640 by height 480 pixels. Photos contained multiple glass jars with jar lids that were intact or damaged. Damage could be subtle or severe, and categories were holes, deformations or scratches. Since there was a lack of training data, these categories were merged to a single category. There are a variety of confounding factors that make it difficult for machine vision to detect defects, such as luminance variation, light reflection and color. Jars were photographed in different positions and were rotated randomly.
Training data consisted of 124 images and validation of 39 images. 5 unseen test images were added as well. Data were annotated using the VGG Image Annotator (VIA) <span style="color:blue">[5]</span>. 

<a id="Data-preprocessing"></a>
<p style="background-color:darkblue;font-family:arial;font-size:120%;color:white;text-align:center;border-radius:2px 2px;"><b>Data preprocessing</b></p>

Running `prepare_annotations()`, first the pandas DataFrame containing the annotation data is loaded. 
A directory structure is created which will contain the image files and label data. Label data are bounding box data normalized between 0 and 1. Data is split into a train and validation set.

In [ ]:
annots = '/kaggle/input/jarlids/jarlids_annots.csv'
imagefiles = '/kaggle/input/jarlids/'
dest = '/kaggle/working/dest/'
results = '/kaggle/working/results.txt'

In [ ]:
yfile='train.yaml'
tr_start=1
tr_end=125
va_start=200
va_end=239
te_start=1
te_end=6
va_te='val.txt'

In [ ]:
class prepare_annotations():
    def __init__(self, sourcedir=annots, imagesdir=imagefiles, destdir=dest, namedir=dest, yamlfile=yfile, filename='im1.jpg', trainstart=tr_start, trainend=tr_end, valstart=va_start, valend=va_end, teststart=te_start, testend=te_end, validationtest=va_te):
        self.sourcedir = sourcedir # directory where annotation data in CSV format are.
        self.imagesdir = imagesdir # directory where original images are.
        self.destdir = destdir # directory where annotations should be put.
        self.namedir = namedir # directory where files are read when Yolov is run; paths are in the .txt and .yaml
        self.yamlfile = yamlfile # name given to the yamlfile 
        self.filename = filename # for ex. p1.jpg
        self.trainstart = trainstart # where train data start
        self.trainend = trainend # where train data end (not including)
        self.valstart = valstart # where validation data start
        self.valend = valend # where validation data end (not including)
        self.teststart = teststart # where testdata start
        self.testend = testend # where testdata end
        self.validationtest = validationtest # name of file used in test, can be 'val.txt' or 'test.txt'
        
    def isize(self, imagesdir, filename):
        """
        Measures image dimensions without loading the image, using the package imagesize.
        imagesdir: path directory to the image files
        filename: name of image file
        """
        w, h = imagesize.get(os.path.join(imagesdir, filename))
        return(w, h)
        
    def load_dataframe(self):
        """
        Loads annotation data with bounding box information and performs preprocessing
        """
        dat = pd.read_csv(self.sourcedir)
        tr = ['p'+str(s)+'.JPG' for s in range(self.trainstart, self.trainend)]
        va = ['p'+str(s)+'.JPG' for s in range(self.valstart, self.valend)]
        te = ['t'+str(s)+'.JPG' for s in range(self.teststart, self.testend)]
        for i,j in zip(['train', 'val', 'test'], [tr, va, te] ):
            dat.loc[dat['filename'].isin(j), 'dataset'] = i
        dat['region_attributes']= dat['region_attributes'].replace({'{}': 'None'})
        dat = dat[~dat['region_attributes'].isin(['None'])].reset_index(drop=False)
        dat['category_names'] = dat['region_attributes'].apply(lambda x: str(list(eval(x).values())[0]))
        dat['category_codes'] = dat[['category_names']].apply(lambda x: pd.Categorical(x).codes)
        dat['image_width'] = dat['filename'].apply(lambda x: self.isize(self.imagesdir, x)[0])
        dat['image_height'] = dat['filename'].apply(lambda x: self.isize(self.imagesdir, x)[1])
        dat['x_min'] = dat['region_shape_attributes'].apply(lambda x: eval(x)['x'])
        dat['y_min'] = dat['region_shape_attributes'].apply(lambda x: eval(x)['y'])
        dat['bb_width'] = dat['region_shape_attributes'].apply(lambda x: eval(x)['width'])
        dat['bb_height'] = dat['region_shape_attributes'].apply(lambda x: eval(x)['height'])
        dat['n_x_center'] = (((dat['x_min'] + dat['bb_width']) + dat['x_min']) / 2) / dat['image_width'] 
        dat['n_y_center'] = (((dat['y_min'] + dat['bb_height']) + dat['y_min']) / 2) / dat['image_height'] 
        dat['n_width'] = dat['bb_width'] / dat['image_width'] 
        dat['n_height'] = dat['bb_height'] / dat['image_height']
        dat['color_cat'] = dat['category_names'].replace({'intact': 'green', 'damaged': 'red'})
        dat = dat.reset_index(drop=True)   
        return(dat)
    
    def make_dirstructure(self):
        """
        Creates directory structure and copies image files
        """
        try:
            print('new directory tree prepared')
            os.makedirs(self.destdir, exist_ok=True)
            os.makedirs(os.path.join(self.destdir, 'images/'), exist_ok=True)
            os.makedirs(os.path.join(self.destdir, 'labels/'), exist_ok=True)
        except:
            print('no new directory was made, probably already existing')
                
        dat = self.load_dataframe()
        filenames = list(set(dat['filename']))
        for f in filenames:
            copyfile(os.path.join(self.imagesdir, f), os.path.join(self.destdir, 'images', f))
                                                
    def make_labels(self):
        """
        Saves bounding box data for given filenames as txt file for each image.
        """
        dat = self.load_dataframe()
        print('length data:', len(dat))
        print('Emptying labelfiles')
        for i in list(set(dat['filename'])):
            try:
                os.remove(os.path.join(self.destdir, 'labels/', i[:-4]+'.txt'))
            except:
                pass

        print('Collecting bounding boxes, saving them to:', os.path.join(self.destdir, 'labels/'))
        for i in range(0,len(dat)):
            try:
                with open(os.path.join(self.destdir, 'labels/', dat['filename'][i][:-4]+'.txt'), "a") as f:
                    print(dat['category_codes'][i], dat['n_x_center'][i], dat['n_y_center'][i], dat['n_width'][i], dat['n_height'][i], file=f)
            except:
                print('something went wrong at', i)

    def fixed_train_val_split(self):
        """
        Images are separated given index. Images that contained the same jarlids but photographed from a different angle were in the same split
        """
        dat = self.load_dataframe()
    
        tr = ['p'+str(i)+'.JPG' for i in range(self.trainstart,self.trainend)]
        va = ['p'+str(i)+'.JPG' for i in range(self.valstart, self.valend)]
        te = ['t'+str(i)+'.JPG' for i in range(self.teststart, self.testend)]
        print('Splitting train and validation data')
        
        try:
            print("Removed previous files")
            os.remove('train.txt')
            os.remove('val.txt')
            os.remove('test.txt')
        except:
            print("No previous files train.txt, val.txt or test.txt were found")
        
        with open(os.path.join(self.destdir, 'train.txt'), "a") as f:
            for i in range(len(tr)):
                print(os.path.join(self.namedir, 'images', tr[i]), file=f)   
        with open(os.path.join(self.destdir, 'val.txt'), "a") as f:
            for i in range(len(va)):
                print(os.path.join(self.namedir, 'images', va[i]), file=f)   
        with open(os.path.join(self.destdir, 'test.txt'), "a") as f:
            for i in range(len(te)):
                print(os.path.join(self.namedir, 'images', te[i]), file=f)   
                
    def yaml_file(self):
        """
        *lb: labels such as 'cat', 'dog'
        namedir: # name directory where Docker Yolov5 reads the files.
        yamlfile: name of the design file 
        validationtest: requires val.txt or test.txt
        """
        dat = self.load_dataframe()
        lb = list(sorted(set(dat['category_names'])))
  
        with open(os.path.join(self.destdir, self.yamlfile), "a") as f:
            print('yaml:', file=f)
            print('names:', file=f)
            for i in lb:
                print('- ', i, file=f)
            print('nc:', len(lb), file=f)
            print('train: ', os.path.join(self.namedir, 'train.txt'), file=f)
            print('val: ', os.path.join(self.namedir, self.validationtest), file=f)

In [ ]:
df = prepare_annotations().load_dataframe()

In [ ]:
df.head()

<a id="Visualization"></a>
<p style="background-color:darkblue;font-family:arial;font-size:120%;color:white;text-align:center;border-radius:2px 2px;"><b>Visualization and annotation boxes</b></p>

Let us visualize the annotations. This can of course also be done with the VGG Image Annotator which was used for annotation <span style="color:blue">[5]</span>.

In [ ]:
def visualisation_annotations(dat, filedir, fname):
    """
    dat: input pandas DataFrame
    filedir: directory where image files can be found
    fname: filename to be visualized, for ex 'p1.JPG'  
    """
    im = Image.open(os.path.join(filedir, fname))
    fig, ax = plt.subplots(figsize=(14, 20))
    ax.imshow(im)
    ndat = dat[dat['filename'] == fname].reset_index()
        
    for i in range(len(ndat)):    
        xmin = ndat['x_min'][i]
        ymin = ndat['y_min'][i]
        w = ndat['bb_width'][i]
        h = ndat['bb_height'][i]
        color = ndat['color_cat'][i]
        rect = patches.Rectangle((xmin, ymin), w, h, linewidth=2, edgecolor=color, facecolor='none') # takes x, y, width and height.
        ax.add_patch(rect)
        
    mpatches_dat = pd.DataFrame(df[['category_names', 'color_cat']].value_counts().reset_index())
    patch1 = mpatches.Patch(color=mpatches_dat['color_cat'][0], label=mpatches_dat['category_names'][0])
    patch2 = mpatches.Patch(color=mpatches_dat['color_cat'][1], label=mpatches_dat['category_names'][1])
    ax.legend(handles=[patch1, patch2])
        
    plt.show()

In [ ]:
visualisation_annotations(df, imagefiles, 'p15.JPG')

<a id="Bounding-box"></a>
<p style="background-color:darkblue;font-family:arial;font-size:120%;color:white;text-align:center;border-radius:2px 2px;"><b>Bounding box size</b></p>

Bounding box size may be a confounding factor and could be predictive in the train and validation set, while it is not in the unseen testdata. As can be seen in the Raincloud plot below, bounding box width and height was similar for intact and damaged jar lids.

In [ ]:
f, ax = plt.subplots(figsize=(12, 8))

pt.RainCloud(x = "category_names", y = "bb_width", data = df[df['dataset'] == 'train'], palette = "Set2", bw = .2,
                 width_viol = .6, ax = ax, orient = "h")

plt.title("Bounding box width between intact and damaged jarlids")

In [ ]:
f, ax = plt.subplots(figsize=(12, 8))

pt.RainCloud(x = "category_names", y = "bb_height", data = df[df['dataset'] == 'train'], palette = "Set2", bw = .2,
                 width_viol = .6, ax = ax, orient = "h")

plt.title("Bounding box height between intact and damaged jarlids")

A scatterplot shows that the dimensions of the bounding box have in general similar aspect ratios.

In [ ]:
plt.figure(figsize=[14,8])
sns.scatterplot(x='bb_width', y='bb_height', data=df.loc[df['dataset'] == 'train'].reset_index(), hue='category_names', alpha=0.7, s=50, palette=dict(intact="#4c9409", damaged="#d48c11"))

<a id="Class-distribution"></a>
<p style="background-color:darkblue;font-family:arial;font-size:120%;color:white;text-align:center;border-radius:2px 2px;"><b>Class distribution</b></p>

There is a slight imbalance between intact and damaged jarlids.

In [ ]:
def count_classes(dat, dataset):
    """
    dat: input DataFrame
    dataset: "train", "val", or "test"
    """
    return(df.loc[df['dataset'] == dataset, "category_names"].value_counts())

Class distribution for train data

In [ ]:
count_classes(df, "train")

Class distribution for validation data

In [ ]:
count_classes(df, "val")

<a id="Preparation-annotations"></a>
<p style="background-color:darkblue;font-family:arial;font-size:120%;color:white;text-align:center;border-radius:2px 2px;"><b>Preparation of annotations</b></p>

We will continue below with the preparation of annotations in the format that is needed for Yolov7. First of all, we start by making a directory structure where the image files and annotations can be stored. 

In [ ]:
prepare_annotations().make_dirstructure()

In [ ]:
prepare_annotations().make_labels()

In [ ]:
prepare_annotations().fixed_train_val_split()

In [ ]:
prepare_annotations().yaml_file()

Now the data structure is ready for training the object detection model.

In [ ]:
cat /kaggle/working/dest/train.yaml

We will look at the annotation data for one image file. The first column displays the category (intact or damaged), the next four columns are the normalized x_center, normalized y_center, normalized width and normalized height of each bounding box.

In [ ]:
cat '/kaggle/working/dest/labels/p1.txt'

<a id="Training"></a>
<p style="background-color:darkblue;font-family:arial;font-size:120%;color:white;text-align:center;border-radius:2px 2px;"><b>Training</b></p>

The model will be trained using data augmentation. In the file `hypownsettings.yaml`, scaling and mosaic are set at 0 since during experiments it turned out that these settings had a negative impact on training. Flip upsidedown is set at 0.5.

In [ ]:
os.environ["WANDB_MODE"]="offline"

In [ ]:
!cp 'yolov7/cfg/training/yolov7.yaml' '/kaggle/working/yolov7_ad.yaml' 

In [ ]:
!cp 'yolov7/cfg/training/yolov7.yaml' '/kaggle/working/yolov7_ad.yaml' 
with open("/kaggle/working/yolov7_ad.yaml") as r:
  text = r.read().replace("nc: 80", "nc: 2")
with open("/kaggle/working/yolov7_ad.yaml", "w") as w:
  w.write(text)

In [ ]:
!head -5 '/kaggle/working/yolov7_ad.yaml'

In [ ]:
cd /kaggle/working/yolov7/

In [ ]:
!cp 'data/hyp.scratch.custom.yaml' '/kaggle/working/hypownsettings.yaml' 
with open("/kaggle/working/hypownsettings.yaml") as r:
  text = r.read().replace("mosaic: 1.0", "mosaic: 0.0").replace("flipud: 0.0", "flipud: 0.5").replace("scale: 0.5", "scale: 0.0")
with open("/kaggle/working/hypownsettings.yaml", "w") as w:
  w.write(text)

We will start training. The flag `--image-weights` is used to correct for class imbalance. Training will benefit from pretrained weights.
This will take some time. You can grab a coffee ☕️ or tea 🫖 in the meanwhile. 

In [ ]:
!python train.py --img 640 --batch 16 --epochs 100 --data /kaggle/working/dest/train.yaml --hyp "/kaggle/working/hypownsettings.yaml" --image-weights --cfg "/kaggle/working/yolov7_ad.yaml" --weights '/kaggle/working/yolov7.pt' --cache

Training gives various files including a `results.txt` file that will in the following line be copied to the Kaggle working space. 

In [ ]:
!cp '/kaggle/working/yolov7/runs/train/exp/results.txt' '/kaggle/working/results.txt'
!cp '/kaggle/working/yolov7/runs/train/exp/results.png' '/kaggle/working/results.png'

In [ ]:
!cp '/kaggle/working/yolov7/runs/train/exp/weights/best.pt' '/kaggle/working/y7-prdef.pt'

<a id="Visualization-learning"></a>
<p style="background-color:darkblue;font-family:arial;font-size:120%;color:white;text-align:center;border-radius:2px 2px;"><b>Visualization of the learning curves</b></p>

To evaluate the results for our purpose, we will look if the bounding boxes are well located around the jar lids and if the predicted class corresponds to the annotated class.
To start the visualization, we will need to replace multiple spaces with commas in the `results.txt` file, so we can display performance below.

In [ ]:
column_names = ['empty', 'x1', 'x2', 'Box', 'Objectness', 'Classification', 'x3', 'x4', 'x5', 'Precision', 'Recall', 'mAP@0.5', 'mAP@0.5:0.95', 'val_Box', 'val_Obj', 'val_Class']

In [ ]:
def text_spaces(inputtext, outputtext, colnames):
    """
    removes spaces and \n from results.txt file and returns data in dataframe
    inputtext: for ex 'results.txt'
    outputtext: for ex 'results.csv'
    colnames: list with columnnames
    """
    dat = pd.DataFrame(columns=colnames)

    with open(inputtext) as f:
        lines = f.readlines()
    print('Length lines:', len(lines))        
    
    for i in range(len(lines)):
        t1 = re.sub(' +', ', ', lines[i])
        t2 = re.sub('\n', '', t1)
        t3 = t2.split(",")
        dat.loc[i] = t3
    dat.to_csv(outputtext)

    return(dat)

In [ ]:
df_lcurves = text_spaces(inputtext=results, outputtext='/kaggle/working/results.txt', colnames=column_names)

In [ ]:
plt.style.use('seaborn-whitegrid')
plt.figure(figsize = (14, 10))

line1 = ['Box', 'Objectness', 'Classification', 'Precision', 'mAP@0.5']
line2 = ['val_Box', 'val_Obj', 'val_Class', 'Recall', 'mAP@0.5:0.95']
col1 = ['blue', 'blue', 'blue', 'green', 'green']
col2 = ['red', 'red', 'red', 'orange', 'orange']
ylab = ['box_loss', 'obj_loss', 'cls_loss', 'recall and precision', 'mAP']
p1 = ['train', 'train', 'train', 'precision', 'mAP_0.5']
p2 = ['val', 'val', 'val', 'recall', 'mAP_0.5:0.95']

df_lcurves[line1] = df_lcurves[line1].astype(float)
df_lcurves[line2] = df_lcurves[line2].astype(float)

for i in range(len(line1)):
    
    ax = plt.subplot(3, 2, i+1)
    ax.plot(df_lcurves[line1[i]], '.-', color=col1[i])
    ax.plot(df_lcurves[line2[i]], '.-', color=col2[i])
    ax.set_ylabel(ylab[i])
    patch1 = mpatches.Patch(color=col1[i], label=p1[i])
    patch2 = mpatches.Patch(color=col2[i], label=p2[i])
    ax.legend(handles=[patch1, patch2])

plt.show()

The mAP (mean Average Precision) is generally used in object detection to evaluate performance. The curves show an increasing trend even though they seem noisy. Other learning curves like bounding box loss are showing good performance as well.

<a id="Unseen-testdata"></a>
<p style="background-color:darkblue;font-family:arial;font-size:120%;color:white;text-align:center;border-radius:2px 2px;"><b>Unseen testdata</b></p>

A few testdata were generated. To test more extensively, a greater variety of images should be used.

In [ ]:
prepare_annotations(yamlfile='test_design.yaml', validationtest='test.txt').yaml_file()

In [ ]:
!python test.py --data /kaggle/working/dest/test_design.yaml --weights '/kaggle/working/yolov7/runs/train/exp/weights/best.pt' --augment

The results will be shown below. Class and bounding box location are quite alright.

In [ ]:
Image.open('/kaggle/working/yolov7/runs/test/exp/test_batch0_labels.jpg')

<a id="Closing-words"></a>
<p style="background-color:darkblue;font-family:arial;font-size:120%;color:white;text-align:center;border-radius:2px 2px;"><b>Closing words</b></p>

The learning curves are showing the desired trends and are going in the good direction. We should remember that the training dataset is rather small, and increasing the size will probably improve the model's generalizability to unseen testdata. This can then be tested for a larger variety of contexts.
Several confounding factors, such as luminance variation, reflectivity, and color, may affect the performance and should be taken into account in order to deploy a machine vision model that can be used in production lines.

Please feel free to check out the model here:
👉 https://huggingface.co/spaces/rrighart/product-defects

![product-defects](https://www.rrighart.com/uploads/8/3/7/7/83774724/gradioapp-product-defects_orig.png)

**References**

[1]. Using deep learning to detect defects in manufacturing. Materials, 13, 5755.

[2]. Xu, Y., Zhang, K., Wang, L. (2021). Metal surface defect detection using modified YOLO. Algorithms.

[3]. Redmon, J., Divvala, S., Girshick, R., Farhadi, A. (2016). You Only Look Once : Unified, real-time object detection. https://arxiv.org/abs/1506.02640 

[4]. YOLOv7: Trainable bag-of-freebies sets new state-of-the-art for real-time object detectors, https://arxiv.org/abs/2207.02696 

[5]. VGG Image Annotator (VIA). https://www.robots.ox.ac.uk/~vgg/software/via/ .

If this notebook helped you, I'd very much appreciate your upvote 😇. 
Please feel free to contact me at rrighart@googlemail.com or get in touch at [LinkedIn](https://www.linkedin.com/in/ruthger-righart).

<li><a href="#Table">Table</a></li>